In [20]:
import os
import random
import numpy as np

In [36]:
# generate mz format corpus
datasets = ["train_2011", "test_2011", "train_2013", "test_2013"]
years = ["2011", "2012", "2013", "2014"]
w2i = {}
i = 0
for dataset, year in zip(datasets, years):
    fn = "/u4/w85yang/deep-tweet-search/data/twitter/order_by_rel/{}".format(dataset)
    # In fact, a.toks does not increase the vocabulary size
    a = open(os.path.join(fn, "a.toks"))
    b = open(os.path.join(fn, "b.toks"))
    c = open(os.path.join(fn, "id.txt"))
    sim = open(os.path.join(fn, "sim.txt"))
    with open("TweetCorpus/{}.txt".format(year), "w") as out:
        for l1, l2, l3, l4 in zip(a, b, c, sim):
            qid, iternum, docno, aid, undefined, run_id = l3[:-1].split()
            out.write("{}\t{}\t{}\t{}\t{}\n".format(l4[:-1], l1[:-1], l2[:-1], qid, docno))

In [37]:
def sample_val(years, test_year, num_samples, val_split=0.1):
    """
    :param datasets: list of training set names to be sampled
    :param num_samples: total number of training samples
    :param val_split: ratio of validation set
    :return: indices of validation samples
    """
    a = open("TweetCorpus/train_{}.txt".format(test_year), "w")
    b = open("TweetCorpus/dev_{}.txt".format(test_year), "w")
    count = num_samples * val_split / 3
    sample_docids = []
    for year in years:
        c = open("/u4/w85yang/RankLib/trunk/data/{}.txt".format(year))
        last_qid = "1"
        qid2docid = []
        for ind, l in enumerate(c):
            qid = l[:0-1].split()[1].split(':')[1]
            if qid != last_qid:
                qid2docid.append(ind)
                last_qid = qid
        qid2docid.append(ind + 1)
        
        sampled_qids = set()
        num = 0
        while (True):
            sample_qids = random.sample(list(range(len(qid2docid))), 1)
            # print("sampled qid: {} in dataset {}".format(sample_qids, data_name))
            qid = sample_qids[0]
            if qid in sampled_qids:
                continue
            sampled_qids.add(qid)
            startInd = qid2docid[qid - 1] if qid != 0 else 0
            endInd = qid2docid[qid]
            num += endInd - startInd
            if num > count * 1.05:
                print("validate {} samples in dataset {}".format(num, year))
                break
            sample_docids.extend(range(startInd, endInd))
            if num > count * 0.95:
                print("validate {} samples in dataset {}".format(num, year))
                break
        
        c = open("TweetCorpus/{}.txt".format(year))
        for ind, l in enumerate(c):
            if ind in sample_docids:
                b.write(l)
            else:
                a.write(l)
    print("validate {} samples in total".format(len(sample_docids)))
    return sample_docids

In [38]:
# generate train-dev sets for each year
years2 = ["2011", "2012", "2013", "2014"]
for test_year in years2:
    random.seed(123456789)
    np.random.seed(123456789)
    years = ["2011", "2012", "2013", "2014"]
    years.remove(test_year)
    dataset2count = {"2011": 39780, "2013": 46192, "2012": 49879, "2014": 41579}
    num_samples = 0
    for year in years:
        num_samples += dataset2count[year]

    sample_docids = sample_val(years, test_year, num_samples)

validate 4622 samples in dataset 2012
validate 4801 samples in dataset 2013
validate 5128 samples in dataset 2014
validate 13739 samples in total
validate 4201 samples in dataset 2011
validate 4651 samples in dataset 2013
validate 4670 samples in dataset 2014
validate 11974 samples in total
validate 4201 samples in dataset 2011
validate 4879 samples in dataset 2012
validate 4670 samples in dataset 2014
validate 12186 samples in total
validate 5111 samples in dataset 2011
validate 4895 samples in dataset 2012
validate 4922 samples in dataset 2013
validate 12204 samples in total
